In [4]:
#Channel handles
handles = ["@ScreenRantPlus", "@CinemaSins", "@DylanIsInTrouble", "@MovieClips"]

In [3]:
#Creating output folder

import os

output_directory = "output"
preserved_directory = "preserved"

os.makedirs(output_directory, exist_ok="TRUE")
os.makedirs(preserved_directory, exist_ok="TRUE")

print("Folders ready.")

Folders ready.
